In [1]:
import os
import numpy as np
import random
import time

def folder_list(path,label):
    '''
    PARAMETER PATH IS THE PATH OF YOUR LOCAL FOLDER
    '''
    filelist = os.listdir(path)
    review = []
    for infile in filelist:
        file = os.path.join(path,infile)
        r = read_data(file)
        r.append(label)
        review.append(r)
    return review

def read_data(file):
    '''
    Read each file into a list of strings.
    Example:
    ["it's", 'a', 'curious', 'thing', "i've", 'found', 'that', 'when', 'willis', 'is', 'not', 'called', 'on',
    ...'to', 'carry', 'the', 'whole', 'movie', "he's", 'much', 'better', 'and', 'so', 'is', 'the', 'movie']
    '''
    f = open(file)
    lines = f.read().split(' ')
    symbols = '${}()[].,:;+-*/&|<>=~" '
    words = map(lambda Element: Element.translate(str.maketrans("", "", symbols)).strip(), lines)
    words = filter(None, words)
    return list(words)


def load_and_shuffle_data():
    '''
    pos_path is where you save positive review data.
    neg_path is where you save negative review data.
    '''
    pos_path = "Data/pos"
    neg_path = "Data/neg"

    pos_review = folder_list(pos_path,1)
    neg_review = folder_list(neg_path,-1)

    review = pos_review + neg_review
    random.shuffle(review)
    return review

# Taken from http://web.stanford.edu/class/cs221/ Assignment #2 Support Code
def dotProduct(d1, d2):
    """
    @param dict d1: a feature vector represented by a mapping from a feature (string) to a weight (float).
    @param dict d2: same as d1
    @return float: the dot product between d1 and d2
    """
    if len(d1) < len(d2):
        return dotProduct(d2, d1)
    else:
        return sum(d1.get(f, 0) * v for f, v in d2.items())

def increment(d1, scale, d2):
    """
    Implements d1 += scale * d2 for sparse vectors.
    @param dict d1: the feature vector which is mutated.
    @param float scale
    @param dict d2: a feature vector.

    NOTE: This function does not return anything, but rather
    increments d1 in place. We do this because it is much faster to
    change elements of d1 in place than to build a new dictionary and
    return it.
    """
    for f, v in d2.items():
        d1[f] = d1.get(f, 0) + v * scale


In [2]:
data = load_and_shuffle_data()

## Question 6

In [3]:
#Create bag of words function
def bag_of_words_func(words):
    """
    Inputs:
    words: (list) a list of words
    
    Output:
    bag_of_words: (dictionary) Key: Word in words - Value: Count of Word in words
    """
    bag_of_words = {}
    #Exclude the last character as it is the label we need to predict
    for word in words[:-1]:
        bag_of_words[word] = bag_of_words.get(word,0) + 1
   
    #Return our bag of words
    return bag_of_words
    

## Question 7

In [4]:
#Grab the first 1500 Reviews / Labels for training
X_train = [bag_of_words_func(f) for f in data[:1500]]
y_train = [f[-1] for f in data[:1500]]

#Grab 500 more for Testing
X_test = [bag_of_words_func(f) for f in data[1500:2000]]
y_test = [f[-1] for f in data[1500:2000]]

## Question 8

In [5]:
def pegasos(x,y, lambda_reg, total_epochs):
    """
    Input:
    x: (dictionary) review data that has been manipulated into sparse dictionary representation
    y: (list) label values of ith position of data at x_i
    lambda_reg
    total_epochs: (int):  
    
    Output:
    w: dictionary of key value pairs, key: word in review, value: float describing the weight of prediction
    """
    
    #Initialize helper variables
    w = dict()
    epoch, t = 0, 0
    
    while epoch < total_epochs:
        for review in range(len(x)):
            
            #Update counter variable
            t += 1
            #Update step size
            step_size_t = 1/(lambda_reg*t)
            
            #Scale w variables
            increment(w, -step_size_t*lambda_reg, w)
            
            #If we have a missclassification, subtract the second portion of the gradient
            if y[review]*dotProduct(w,x[review]) < 1:
                increment(w, step_size_t*y[review], x[review])
        #Increment epoch counter variable
        epoch += 1     
        
    return w

In [6]:
def fast_pegasos_algo(x,y,lambda_reg,total_epochs):
    """
    Input:
    x: (dictionary) review data that has been manipulated into sparse dictionary representation
    y: (list) label values of ith position of data at x_i
    lambda_reg
    total_epochs: (int):  
    
    Output:
    w: dictionary of key value pairs, key: word in review, value: float describing the weight of prediction
    """
    
    #Initialize helper variables
    W = dict()
    epoch, t, s = 0, 1, 1
    
    while epoch < total_epochs:
        for review in range(len(x)):
            
            #Update counter variable
            t += 1
            #Update step size
            step_size_t = 1/(lambda_reg*t)
            
            #Update s
            s += (s * lambda_reg * -step_size_t)
            
            #If we have a missclassification, subtract the second portion of the gradient
            if y[review]*dotProduct(W,x[review])*s < 1:
                increment(W, (1/s)*step_size_t*y[review], x[review])
        #Increment epoch counter variable
        epoch += 1     
    W.update((x,y*s) for x,y in W.items())   
    return W

## Problem 10

In [7]:
lambda_reg = .1
epochs = 6

In [8]:
#Time test each approach
start = time.time()
w_slow = pegasos(X_train,y_train,lambda_reg, epochs)
end = time.time()
print('Slow pegasos algorithm run speed:', end-start)

start = time.time()
w_fast = fast_pegasos_algo(X_train,y_train,lambda_reg, epochs)
end = time.time()
print('Fast pegasos algorithm run speed:', end-start)

Slow pegasos algorithm run speed: 42.33915710449219
Fast pegasos algorithm run speed: 0.6696586608886719


## Problem 11

In [9]:
def classification_error(x,y,w):
    total_error = 0
    
    #Iterate over data
    for row in range(len(x)):
        #Get prediction
        if dotProduct(x[row],w)<0:
            y_hat = -1
        else:
            y_hat = 1
        
        if y_hat != y[row]:
            total_error += 1
    
    
    return total_error / len(y)

In [10]:
print('Slow loss:', classification_error(X_test,y_test,w_slow) , 'Fast loss:',classification_error(X_test,y_test,w_fast))

Slow loss: 0.174 Fast loss: 0.174


## Problem 12

In [11]:
error_dict = dict()

for i in range(15):
    temp = 10**(-i)
    label = str(temp)
    w_fast = fast_pegasos_algo(X_train,y_train, temp, 6)
    error_dict[label] = classification_error(X_test,y_test,w_fast)
    

In [12]:
error_dict

{'1': 0.196,
 '0.1': 0.174,
 '0.01': 0.15,
 '0.001': 0.166,
 '0.0001': 0.188,
 '1e-05': 0.188,
 '1e-06': 0.188,
 '1e-07': 0.188,
 '1e-08': 0.188,
 '1e-09': 0.188,
 '1e-10': 0.188,
 '1e-11': 0.188,
 '1e-12': 0.188,
 '1e-13': 0.188,
 '1e-14': 0.188}

In [23]:
error_dict = dict()

for i in np.linspace(.001,.01,50):
    label = str(i)
    w_fast = fast_pegasos_algo(X_train,y_train, i, 6)
    error_dict[label] = classification_error(X_test,y_test,w_fast)
    

In [28]:
min(error_dict.values())

0.142